Clone github and download models

In [1]:
!git clone https://github.com/sberbank-ai/sber-swap.git
%cd sber-swap

# load arcface
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py

# load landmarks detector
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx

# load model itself
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth

# load super res model
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth

Cloning into 'sber-swap'...
remote: Enumerating objects: 704, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 704 (delta 41), reused 21 (delta 21), pack-reused 645
Receiving objects: 100% (704/704), 94.00 MiB | 21.80 MiB/s, done.
Resolving deltas: 100% (168/168), done.
/content/sber-swap
--2022-10-27 18:14:17--  https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ai-forever/ghost/releases/download/arcface/backbone.pth [following]
--2022-10-27 18:14:17--  https://github.com/ai-forever/ghost/releases/download/arcface/backbone.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-producti

Required libraries

In [2]:
!pip install mxnet-cu112
!pip install onnxruntime-gpu==1.8
!pip install insightface==0.2.1
!pip install kornia==0.5.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 499.4 MB 1.6 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.3 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 14.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 285 kB 14.9 MB/s 


In [3]:
import cv2
import torch
import time
import os

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,
/usr/local/lib/python3.7/dist-packages/mxnet/numpy/utils.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bool = onp.bool
/usr/local/lib/python3.7/dist-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr

In [4]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
Network [LIPSPADEGenerator] was created. Total number of parameters: 72.2 million. To see the architecture, do print(network).
Load checkpoint from path:  weights/10_net_G.pth


In [12]:
# for ariana
target_type = 'image'
source_image = os.listdir('/content/drive/MyDrive/CelebA-100')

for i in range(0,len(source_image)):
  bad_good = True
  source_path = '/content/drive/MyDrive/CelebA-100/' + source_image[i]
  target_path = '/content/drive/MyDrive/target_images/ariana.jpg' 
  source_full = cv2.imread(source_path)
  crop_size = 224 # don't change this
  try:
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    bad_good = True
  except:
    bad_good = False
  if bad_good == True:
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)
    batch_size = 40
    final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,source,target,netArc,G,app,set_target = False,crop_size=crop_size,BS=batch_size)
    final_frames_list = face_enhancement(final_frames_list, model)
    result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
    cv2.imwrite('/content/drive/MyDrive/GHOST/Results_ariana/'+source_image[i], result)
  
  print('[Finish]',i, source_image[i])


100%|██████████| 1/1 [00:00<00:00, 69.73it/s]
1it [00:00, 225.42it/s]
1it [00:00, 278.84it/s]
100%|██████████| 1/1 [00:00<00:00, 270.20it/s]
1it [00:00,  8.75it/s]


[Finish] 0 4-O.jpg


100%|██████████| 1/1 [00:00<00:00, 64.96it/s]
1it [00:00, 166.65it/s]
1it [00:00, 2062.10it/s]
100%|██████████| 1/1 [00:00<00:00, 8208.03it/s]
1it [00:00,  8.48it/s]


[Finish] 1 4-P.jpg


100%|██████████| 1/1 [00:00<00:00, 67.84it/s]
1it [00:00, 141.88it/s]
1it [00:00, 1274.86it/s]
100%|██████████| 1/1 [00:00<00:00, 7653.84it/s]
1it [00:00,  8.50it/s]


...

[Finish] 197 62-P.jpg


100%|██████████| 1/1 [00:00<00:00, 44.34it/s]
1it [00:00, 102.82it/s]
1it [00:00, 1793.97it/s]
100%|██████████| 1/1 [00:00<00:00, 9664.29it/s]
1it [00:00,  8.48it/s]


[Finish] 198 90-O.jpg


100%|██████████| 1/1 [00:00<00:00, 63.76it/s]
1it [00:00, 259.15it/s]
1it [00:00, 2033.11it/s]
100%|██████████| 1/1 [00:00<00:00, 8542.37it/s]
1it [00:00,  8.55it/s]


[Finish] 199 90-P.jpg


In [11]:
# for kobe
target_type = 'image'
source_image = os.listdir('/content/drive/MyDrive/CelebA-100')

for i in range(0,len(source_image)):
  bad_good = True
  source_path = '/content/drive/MyDrive/CelebA-100/' + source_image[i]
  target_path = '/content/drive/MyDrive/target_images/kobe.jpg' 
  source_full = cv2.imread(source_path)
  crop_size = 224 # don't change this
  try:
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    bad_good = True
  except:
    bad_good = False
  if bad_good == True:
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)
    batch_size = 40
    final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,source,target,netArc,G,app,set_target = False,crop_size=crop_size,BS=batch_size)
    final_frames_list = face_enhancement(final_frames_list, model)
    result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
    cv2.imwrite('/content/drive/MyDrive/GHOST/Results_kobe/'+source_image[i], result)
  
  print('[Finish]',i, source_image[i])

100%|██████████| 1/1 [00:00<00:00, 61.42it/s]
1it [00:00, 143.65it/s]
1it [00:00, 2014.56it/s]
100%|██████████| 1/1 [00:00<00:00, 11554.56it/s]
1it [00:00,  8.69it/s]


[Finish] 0 4-O.jpg


100%|██████████| 1/1 [00:00<00:00, 64.06it/s]
1it [00:00, 269.82it/s]
1it [00:00, 3243.85it/s]
100%|██████████| 1/1 [00:00<00:00, 11066.77it/s]
1it [00:00,  8.11it/s]


[Finish] 1 4-P.jpg


100%|██████████| 1/1 [00:00<00:00, 71.83it/s]
1it [00:00, 201.18it/s]
1it [00:00, 406.46it/s]
100%|██████████| 1/1 [00:00<00:00, 9279.43it/s]
1it [00:00,  8.31it/s]


...

[Finish] 197 62-P.jpg


100%|██████████| 1/1 [00:00<00:00, 59.62it/s]
1it [00:00, 285.23it/s]
1it [00:00, 1551.15it/s]
100%|██████████| 1/1 [00:00<00:00, 7169.75it/s]
1it [00:00,  8.45it/s]


[Finish] 198 90-O.jpg


100%|██████████| 1/1 [00:00<00:00, 56.99it/s]
1it [00:00, 248.92it/s]
1it [00:00, 1970.08it/s]
100%|██████████| 1/1 [00:00<00:00, 7121.06it/s]
1it [00:00,  7.84it/s]


[Finish] 199 90-P.jpg


In [10]:
# for obama
target_type = 'image'
source_image = os.listdir('/content/drive/MyDrive/CelebA-100')

for i in range(0,len(source_image)):
  bad_good = True
  source_path = '/content/drive/MyDrive/CelebA-100/' + source_image[i]
  target_path = '/content/drive/MyDrive/target_images/obama.jpg' 
  source_full = cv2.imread(source_path)
  crop_size = 224 # don't change this
  try:
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    bad_good = True
  except:
    bad_good = False
  if bad_good == True:
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)
    batch_size = 40
    final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,source,target,netArc,G,app,set_target = False,crop_size=crop_size,BS=batch_size)
    final_frames_list = face_enhancement(final_frames_list, model)
    result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
    cv2.imwrite('/content/drive/MyDrive/GHOST/Results_obama/'+source_image[i], result)
  
  print('[Finish]',i, source_image[i])

100%|██████████| 1/1 [00:00<00:00, 63.40it/s]
1it [00:00, 310.67it/s]
1it [00:00, 2528.21it/s]
100%|██████████| 1/1 [00:00<00:00, 10951.19it/s]
1it [00:00,  8.69it/s]


[Finish] 0 4-O.jpg


100%|██████████| 1/1 [00:00<00:00, 58.97it/s]
1it [00:00, 237.25it/s]
1it [00:00, 2051.00it/s]
100%|██████████| 1/1 [00:00<00:00, 10727.12it/s]
1it [00:00,  8.71it/s]


[Finish] 1 4-P.jpg


100%|██████████| 1/1 [00:00<00:00, 62.74it/s]
1it [00:00, 134.30it/s]
1it [00:00, 1242.76it/s]
100%|██████████| 1/1 [00:00<00:00, 10894.30it/s]
1it [00:00,  8.13it/s]


...

[Finish] 197 62-P.jpg


100%|██████████| 1/1 [00:00<00:00, 67.45it/s]
1it [00:00, 179.44it/s]
1it [00:00, 2104.52it/s]
100%|██████████| 1/1 [00:00<00:00, 9137.92it/s]
1it [00:00,  9.30it/s]


[Finish] 198 90-O.jpg


100%|██████████| 1/1 [00:00<00:00, 62.64it/s]
1it [00:00, 126.30it/s]
1it [00:00, 1683.78it/s]
100%|██████████| 1/1 [00:00<00:00, 8144.28it/s]
1it [00:00,  8.85it/s]


[Finish] 199 90-P.jpg
